In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

# import the parts of RDKit that we need
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
from rdkit import DataStructs
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDepictor, rdMolDescriptors


from rdkit.Chem.Draw import IPythonConsole


import time
rdDepictor.SetPreferCoordGen(True)
print(rdkit.__version__)


2020.03.1


In [2]:
from utils import get_smiles_encodings, load_data, smile_to_hot
file_data = './dataset/data.csv'

# View of the data

In [4]:
df = pd.read_csv(file_data, delimiter=",")

In [6]:
df.head()

,Chiral_Molecular_SMILES,Energy_(kcal/mol),Zero_point_energy_(kcal/mol@0K),Enthalpy_(kcal/mol@298K),Gibbs_energy_(kcal/mol@298K),Energy DG:kcal/mol),PMI1,PMI2,PMI3,NPR1,...,Asphericity,Eccentricity,Core_Core_Repulsion_(kcal/mol),Core_Hamiltonian_(kcal/mol),Two_Electron_Integral_(kcal/mol),Electronic_Energy_(kcal/mol),Heat_of_Formation_(kcal/mol),HOMO_Energy__(eV),LUMO_Energy_(eV),H-L_Gap_(eV)
0,Fc1ccc(cc1)c1cn[nH]c1N,-51177.14356,93.711236,101.010221,72.646019,-51104.49754,15.11691847,1138.689784,1276.446124,0.152022697,...,0.611913907,0.988377003,209590.8432,-82616.31512,-356305.337,-260768.9836,19.36970138,-0.325428768,-0.006221411,0.319207358
1,Fc1ccc(cc1)c1cn[nH]c1N,-51177.14356,93.716622,101.010607,72.676293,-51104.46727,55.76038389,2320.928165,2603.438182,0.277876513,...,0.38663113,0.960615584,423156.7216,-118968.2919,-755464.2531,-496700.4184,-63.1251924,-0.357173861,-0.031289724,0.325884137
2,O=C(c1ccccc1)O[C@H]1c2ccccc2C(=O)N1C,-73542.31526,150.283189,161.552872,122.934889,-73419.38037,16.72360603,1358.577969,1573.221539,0.161400099,...,0.592709472,0.986863133,202454.7033,-81529.04355,-343358.529,-253208.308,11.84605445,-0.326916674,-0.027046341,0.299870332
3,O=C(c1ccccc1)O[C@H]1c2ccccc2C(=O)N1C,-73542.31444,150.296914,161.555511,123.082139,-73419.23230,35.72547442,1045.051233,1174.934149,0.400794729,...,0.235943916,0.914902573,350422.3801,-94499.59329,-629142.5208,-409070.8537,-94.22818817,-0.342691773,0.007354553,0.350046326
4,O=C(c1ccccc1)O[C@@H]1c2ccccc2C(=O)N1C,-73542.31445,150.299719,161.554575,123.129949,-73419.18450,38.86168886,1717.11819,1838.665112,0.270299478,...,0.402459166,0.962733429,343048.4259,-101348.4649,-609028.2103,-405862.5701,-38.21830885,-0.364204826,-0.010250894,0.353953932


In [106]:
df_lowE = df.loc[df.groupby('Chiral_Molecular_SMILES')["Energy_(kcal/mol)"].idxmin()].reset_index(drop=True)
df_lowE

,Chiral_Molecular_SMILES,Energy_(kcal/mol),Zero_point_energy_(kcal/mol@0K),Enthalpy_(kcal/mol@298K),Gibbs_energy_(kcal/mol@298K),Energy DG:kcal/mol),PMI1,PMI2,PMI3,NPR1,...,Asphericity,Eccentricity,Core_Core_Repulsion_(kcal/mol),Core_Hamiltonian_(kcal/mol),Two_Electron_Integral_(kcal/mol),Electronic_Energy_(kcal/mol),Heat_of_Formation_(kcal/mol),HOMO_Energy__(eV),LUMO_Energy_(eV),H-L_Gap_(eV)
0,BrC(=C(C(=O)O)Br)C(=O)O,-49321.20289,34.715581,42.059286,11.299203,-49309.90369,24.7663478,1869.020567,2131.048963,0.224519831,...,0.471984286,0.974469485,294897.2941,-131324.3286,-486513.3035,-374580.9803,-262.9185504,-0.394260331,-0.005083064,0.389177267
1,BrC(=Cc1ccccc1)Br,-34964.62885,67.227058,73.562672,45.951991,-34918.67685,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BrC(=O)c1ccccc1,-33298.59827,58.969189,63.659831,40.882120,-33257.71614,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BrC([N+](=O)[O-])([N+](=O)[O-])[N+](=O)[O-],-60616.48803,27.699422,34.576448,6.069568,-60610.41846,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BrC1=CC(=O)N(C1=O)C,-38198.69914,49.906725,55.916049,29.316249,-38169.38289,20.30058899,835.682829,908.1003598,0.298843893,...,0.360426838,0.954216358,229209.4349,-75556.22275,-401311.2746,-276211.8601,-13.10986577,-0.341259083,0.004599268,0.345858352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12860,n1ccc(cc1)c1nc2c([nH]1)cccc2,-49089.12971,109.626424,116.447601,89.419082,-48999.71063,14.26351248,566.3846649,748.2662913,0.243070732,...,0.448037945,0.970008567,172459.1368,-69595.84494,-291597.9007,-215394.7953,-2.421271855,-0.370916756,-0.039817512,0.331099244
12861,n1ccc(cc1)c1nnc[nH]1,-38162.89848,74.889776,79.857790,56.868521,-38106.02995,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12862,n1ccc(cc1)c1nnn[nH]1,-39143.35213,68.192941,73.049959,50.213757,-39093.13837,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12863,n1ccc(cc1)n1cncc1,-37168.91672,81.715582,87.320224,62.501697,-37106.41503,36.78971745,983.6318461,1283.58284,0.415749931,...,0.218047382,0.909458775,282986.9534,-101083.1434,-487114.8755,-344640.5812,-132.009023,-0.352489769,-0.058200092,0.294289677


In [98]:
a = 0
data = []
columns_with_nan = []
for col in df_lowE.columns : 
    i = 0
    for j in df_lowE[col] : 
        data.append(j)
        print(j)
        i += 1
        if i == 100 : 
            break
        if j == None : 
            print(col)
            columns_with_nan.append(col)
            break

BrC(=C(C(=O)O)Br)C(=O)O
BrC(=Cc1ccccc1)Br
BrC(=O)c1ccccc1
BrC([N+](=O)[O-])([N+](=O)[O-])[N+](=O)[O-]
BrC1=CC(=O)N(C1=O)C
BrC1=CC(=O)N(C1=O)c1ccccc1
BrC1=Cc2c(C1)cccc2
BrC1CCS(=O)(=O)CC1
BrCBr
BrCC(=O)C(C)(C)C
BrCC(=O)CS(=O)(=O)c1ccc(cc1)C
BrCC(=O)NCC#C
BrCC(=O)Nc1cccc(c1)C#C
BrCC(=O)c1c(C)[nH]c2c1cccc2
BrCC(=O)c1c(C)onc1c1ccccc1
BrCC(=O)c1c[nH]c2c1cccn2
BrCC(=O)c1ccc(c(c1)C(=O)N)O
BrCC(=O)c1ccc(c(c1)C)Cl
BrCC(=O)c1ccc(c(c1)F)F
BrCC(=O)c1ccc(cc1)C
BrCC(=O)c1ccc(cc1)NS(=O)(=O)C
BrCC(=O)c1ccc(cc1)OC(F)(F)F
BrCC(=O)c1ccc(cc1)c1ccccc1
BrCC(=O)c1ccc(cc1)c1ccsc1
BrCC(=O)c1ccc(cc1OC)OC
BrCC(=O)c1ccc(s1)c1ccccc1
BrCC(=O)c1ccc2c(c1)CC(=O)N2
BrCC(=O)c1ccc2c(c1)cccc2
BrCC(=O)c1cccc(c1)C#N
BrCC(=O)c1cccc2c1OCCO2
BrCC(=O)c1ccccc1
BrCC(=O)c1cccs1
BrCC(=O)c1ccnc(c1)Cl
BrCC(=O)c1cn(c2c1cccc2)C
BrCC(=O)c1cscc1
BrCC(=O)c1nccs1
BrCC(=O)c1oc2c(c1C)cccc2
BrCC(=O)c1sc(nc1C)c1ccccc1
BrCC(Br)(F)F
BrCC(C(=O)O)CBr
BrCC(C(F)F)(F)F
BrCC(C)C
BrCC1(CC1)CS(=O)(=O)C
BrCCCCC(=O)O
BrCCCCCBr
BrCCCN1C(=O)c2c(C1=O)cccc2
B

nan
-378427.8418
nan
-569274.9429
nan
nan
nan
-252978.9782
nan
nan
-422656.3941
-223809.1388
nan
nan
nan
nan
nan
nan
nan
-429918.3726
nan
-278107.9096
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
-364444.5691
nan
nan
 
nan
-342948.2101
nan
-313211.179
nan
-542301.4782
-278645.1847
nan
nan
nan
-150493.7156
-285268.8547
nan
-143997.7697
nan
nan
nan
nan
nan
nan
-178115.3032
nan
nan
nan
 
nan
nan
nan
nan
nan
nan
nan
-237775.3415
nan
nan
-354870.9328
nan
nan
-271020.6213
nan
-378220.5481
-272319.2291
nan
nan
nan
-262.9185504
nan
nan
nan
-13.10986577
nan
nan
-136.6764533
nan
-29.66540758
nan
nan
nan
6.604140926
nan
nan
-3.578529706
-37.00053187
nan
nan
nan
nan
nan
nan
nan
25.37108544
nan
-53.11146611
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
-43.71302184
nan
nan
 
nan
-29.83185349
nan
-28.2834417
nan
-11.64957714
-42.85861008
nan
nan
nan
6.257692628
62.86159013
nan
-39.8697278
nan
nan
nan
nan
nan

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [105]:
df_lowE.dropna()

,Chiral_Molecular_SMILES,Energy_(kcal/mol),Zero_point_energy_(kcal/mol@0K),Enthalpy_(kcal/mol@298K),Gibbs_energy_(kcal/mol@298K),Energy DG:kcal/mol),PMI1,PMI2,PMI3,NPR1,...,Asphericity,Eccentricity,Core_Core_Repulsion_(kcal/mol),Core_Hamiltonian_(kcal/mol),Two_Electron_Integral_(kcal/mol),Electronic_Energy_(kcal/mol),Heat_of_Formation_(kcal/mol),HOMO_Energy__(eV),LUMO_Energy_(eV),H-L_Gap_(eV)
0,BrC(=C(C(=O)O)Br)C(=O)O,-49321.20289,34.715581,42.059286,11.299203,-49309.90369,24.7663478,1869.020567,2131.048963,0.224519831,...,0.471984286,0.974469485,294897.2941,-131324.3286,-486513.3035,-374580.9803,-262.9185504,-0.394260331,-0.005083064,0.389177267
4,BrC1=CC(=O)N(C1=O)C,-38198.69914,49.906725,55.916049,29.316249,-38169.38289,20.30058899,835.682829,908.1003598,0.298843893,...,0.360426838,0.954216358,229209.4349,-75556.22275,-401311.2746,-276211.8601,-13.10986577,-0.341259083,0.004599268,0.345858352
7,BrC1CCS(=O)(=O)CC1,-39688.78412,82.089287,88.022304,62.671672,-39626.11245,43.02930967,1559.480992,2188.410054,0.304958039,...,0.360511843,0.952206957,311799.4765,-108520.0144,-539815.6548,-378427.8418,-136.6764533,-0.343589097,-0.04865337,0.294935726
9,BrCC(=O)C(C)(C)C,-32393.14214,91.355565,98.082582,71.025066,-32322.11707,69.89640926,2784.569792,3298.286364,0.282932003,...,0.378157892,0.959052823,488478.123,-130519.0886,-877511.7086,-569274.9429,-29.66540758,-0.322268832,-0.028625231,0.293643601
13,BrCC(=O)c1c(C)[nH]c2c1cccc2,-50314.54505,108.114439,116.571108,84.663002,-50229.88205,24.28367362,1047.289075,1156.193763,0.280587996,...,0.38816753,0.958667284,206619.2057,-74513.98441,-356929.9876,-252978.9782,6.604140926,-0.358035522,-0.001907553,0.356127969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12847,c1ccc2c(c1)nsn2,-30452.54767,53.695949,58.090025,36.557013,-30415.99065,53.01549012,2419.982931,3095.918147,0.25087154,...,0.43115969,0.968020377,393087.1472,-111541.9258,-702371.1634,-462727.5075,-9.910930756,-0.32128376,-0.03778658,0.28349718
12849,c1cnc2c(c1)cc[nH]2,-29904.04596,68.924769,73.255648,52.192833,-29851.85313,66.47681514,1753.798154,2487.641648,0.412755982,...,0.235396612,0.909814787,419504.4159,-133776.1612,-734218.5725,-500885.4474,-203.7325193,-0.356460924,-0.026168525,0.330292399
12856,c1nccnc1C#N,-27429.77317,44.488904,48.542341,27.814072,-27401.95909,26.22589468,1869.287803,2198.948131,0.266366285,...,0.402476792,0.963871881,282166.3277,-104722.652,-483337.324,-346391.314,-106.9881186,-0.374795012,-0.059747,0.315048012
12860,n1ccc(cc1)c1nc2c([nH]1)cccc2,-49089.12971,109.626424,116.447601,89.419082,-48999.71063,14.26351248,566.3846649,748.2662913,0.243070732,...,0.448037945,0.970008567,172459.1368,-69595.84494,-291597.9007,-215394.7953,-2.421271855,-0.370916756,-0.039817512,0.331099244
